In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Resizing,Rescaling,Conv2D,Flatten,BatchNormalization,MaxPool2D,Dropout,InputLayer,Dense
from keras.regularizers import L2
from keras.metrics import CategoricalAccuracy ,TopKCategoricalAccuracy
from keras.optimizers import Adam
import cv2

2024-03-31 07:25:57.227585: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-31 07:25:57.227693: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-31 07:25:57.382188: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [52]:
train_directory="/kaggle/input/human-emotions-datasethes/Emotions Dataset/Emotions Dataset/train"
validation_directory="/kaggle/input/human-emotions-datasethes/Emotions Dataset/Emotions Dataset/test"
CLASS_NAMES=["angry","happy","sad"]
CONFIGURATION={
        "BATCH_SIZE":32,
        "IMG_SIZE":256,
        "REGULARIZATION_RATE": 0.0,
      "N_FILTERS": 6,
      "KERNEL_SIZE": 3,
      "N_STRIDES": 1,
      "POOL_SIZE": 2,
      "N_DENSE_1": 100,
      "N_DENSE_2": 10,
    "DROPOUT_RATE": 0.0,
    "NUM_CLASSES":3,
    "LEARNING_RATE":0.001,
    "EPOCHS":10

}

In [53]:
train_dataset=tf.keras.preprocessing.image_dataset_from_directory(
    train_directory,
    labels='inferred',
    label_mode='categorical',
    class_names=CLASS_NAMES,
    color_mode='rgb',
    batch_size=CONFIGURATION["BATCH_SIZE"],
    image_size=(CONFIGURATION["IMG_SIZE"],CONFIGURATION["IMG_SIZE"]),
    shuffle=True,
    seed=42,
  
    
)


Found 6799 files belonging to 3 classes.


In [54]:
val_dataset=tf.keras.preprocessing.image_dataset_from_directory(
    validation_directory,
    labels='inferred',
    label_mode='categorical',
    class_names=CLASS_NAMES,
    color_mode='rgb',
    batch_size=CONFIGURATION["BATCH_SIZE"],
    image_size=(CONFIGURATION["IMG_SIZE"],CONFIGURATION["IMG_SIZE"]),
    shuffle=True,
    seed=42,
)


Found 2278 files belonging to 3 classes.


In [ ]:
for i in val_dataset.take(1):
    print(i)

## <b> TF Records <b>

In [5]:
from tensorflow.train import BytesList,FloatList,Int64List
from tensorflow.train import Example,Features,Feature

In [55]:
training_dataset=(
train_dataset
.unbatch())

In [56]:
validation_dataset=(
    val_dataset.unbatch()
)

In [74]:
def create_example(image,label):
    bytes_feature=Feature(
        bytes_list=BytesList(value=[image])
    )

    int_feature=Feature(
        int64_list=Int64List(value=[np.argmax(label)])
    )
    example=Example(
    features=Features(
    feature={
        'images':bytes_feature,
        'labels':int_feature,

    }
    )
    )
    return example.SerializeToString()

In [20]:
!mkdir tfrecords

In [46]:
NUM_SHARDS =10
PATH="/kaggle/working/tfrecords/shard_{:02d}.tfrecord"

In [93]:
def encode_image(image,label):
    image=tf.image.convert_image_dtype(image,dtype=tf.uint8)
    image=tf.io.encode_jpeg(image)
    return image,tf.argmax(label)
    

In [94]:
encoded_dataset=(
    training_dataset.map(encode_image)
)

In [95]:
for shard_number in range(NUM_SHARDS):
    sharded_dataset=(
    encoded_dataset.shard(NUM_SHARDS,shard_number)
    .as_numpy_iterator())
    with tf.io.TFRecordWriter(PATH.format(shard_number)) as file_writer:
        for image,label in sharded_dataset:
            example=create_example(image,label)
            file_writer.write(example)


In [50]:
encoded_dataset

{<_MapDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(3,), dtype=tf.float32, name=None))>}

In [96]:
recons_dataset=tf.data.TFRecordDataset(
    filenames=[PATH.format(p) for p in range(NUM_SHARDS
                                            )]
)

In [87]:
l=[PATH.format(p) for p in range(NUM_SHARDS)]
print(l)

['/kaggle/working/tfrecords/shard_00.tfrecord', '/kaggle/working/tfrecords/shard_01.tfrecord', '/kaggle/working/tfrecords/shard_02.tfrecord', '/kaggle/working/tfrecords/shard_03.tfrecord', '/kaggle/working/tfrecords/shard_04.tfrecord', '/kaggle/working/tfrecords/shard_05.tfrecord', '/kaggle/working/tfrecords/shard_06.tfrecord', '/kaggle/working/tfrecords/shard_07.tfrecord', '/kaggle/working/tfrecords/shard_08.tfrecord', '/kaggle/working/tfrecords/shard_09.tfrecord']


In [97]:
def parse_tfrecords(example):
    feature_description={
        "images":tf.io.FixedLenFeature([],tf.string),
        "labels":tf.io.FixedLenFeature([],tf.int64),
        
    }
    example=tf.io.parse_single_example(example,feature_description)
    example['images']=tf.io.decode_jpeg(example['images'],channels=3)
    return example['images'],example['labels']

In [98]:
parsed_datasets=(
    recons_dataset.map(parse_tfrecords).batch(CONFIGURATION["BATCH_SIZE"])
    .prefetch(tf.data.AUTOTUNE)
)

In [100]:
parsed_datasets

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [101]:
for i in parsed_datasets.take(1):
    print(i)

(<tf.Tensor: shape=(32, 256, 256, 3), dtype=uint8, numpy=
array([[[[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],

        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],

        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],

        ...,

        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],

        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],

        [[255, 255, 255],
         [255, 255, 255],
         [255, 2

In [92]:
parsed_datasets

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [10]:
# Assuming train_dataset is a TensorFlow dataset object and it has been unbatched
for example in train_dataset.take(1): # Printing first 5 examples
    print(example)


(<tf.Tensor: shape=(32, 256, 256, 3), dtype=float32, numpy=
array([[[[189.        , 189.        , 189.        ],
         [189.8125    , 189.8125    , 189.8125    ],
         [198.9375    , 198.9375    , 198.9375    ],
         ...,
         [122.9375    , 122.9375    , 122.9375    ],
         [147.        , 147.        , 147.        ],
         [147.        , 147.        , 147.        ]],

        [[188.1875    , 188.1875    , 188.1875    ],
         [190.32031   , 190.32031   , 190.32031   ],
         [199.1914    , 199.1914    , 199.1914    ],
         ...,
         [127.96484   , 127.96484   , 127.96484   ],
         [146.39062   , 146.39062   , 146.39062   ],
         [143.75      , 143.75      , 143.75      ]],

        [[168.75      , 168.75      , 168.75      ],
         [170.07031   , 170.07031   , 170.07031   ],
         [168.22656   , 168.22656   , 168.22656   ],
         ...,
         [143.13281   , 143.13281   , 143.13281   ],
         [137.14062   , 137.14062   , 137.1406

# Dataset Visualization

In [ ]:
plt.figure(figsize=(12,12))
for images,labels in train_dataset.take(1):
    for i in range(16):
        po=plt.subplot(4,4,i+1)
        plt.imshow(images[i]/255.)
        plt.title(CLASS_NAMES[tf.argmax(labels[i],axis=0).numpy()])
        plt.axis("off")
        
        

In [ ]:
train_dataset=(
    train_dataset.prefetch(tf.data.AUTOTUNE)
)

#### > So, in summary, the line train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE) is optimizing your training data pipeline by prefetching batches of data in the background while the model is training, potentially reducing training time by keeping the GPU or CPU busy with data processing tasks.

In [ ]:
validation_dataset=(
    val_dataset.prefetch(tf.data.AUTOTUNE)
)

In [ ]:
resize_rescale_layers=tf.keras.Sequential([
    Resizing(CONFIGURATION["IMG_SIZE"],CONFIGURATION["IMG_SIZE"]),
    Rescaling(1./255)
])

# <b>Modeling<b>
    

In [ ]:
DROPOUT_RATE = CONFIGURATION['DROPOUT_RATE']
REGULARIZATION_RATE = CONFIGURATION['REGULARIZATION_RATE']
N_FILTERS = CONFIGURATION['N_FILTERS']
KERNEL_SIZE = CONFIGURATION['KERNEL_SIZE']
POOL_SIZE = CONFIGURATION['POOL_SIZE']
N_STRIDES = CONFIGURATION['N_STRIDES']
lenet_model=tf.keras.Sequential(
[
    InputLayer(input_shape=(None,None,3)),
    resize_rescale_layers,
    Conv2D(filters = N_FILTERS , kernel_size = KERNEL_SIZE, strides = N_STRIDES , padding='valid',
          activation = 'relu',kernel_regularizer = L2(REGULARIZATION_RATE)),
    BatchNormalization(),
    MaxPool2D (pool_size = POOL_SIZE, strides= N_STRIDES*2),
    Dropout(rate = DROPOUT_RATE ),

    Conv2D(filters = N_FILTERS*2 + 4, kernel_size = KERNEL_SIZE, strides=N_STRIDES, padding='valid',
          activation = 'relu', kernel_regularizer = L2(REGULARIZATION_RATE)),
    BatchNormalization(),
    MaxPool2D (pool_size = POOL_SIZE, strides= N_STRIDES*2),

    Flatten(),
    
    Dense( CONFIGURATION['N_DENSE_1'], activation = "relu", kernel_regularizer = L2(REGULARIZATION_RATE)),
    BatchNormalization(),
    Dropout(rate = DROPOUT_RATE),
    
    Dense( CONFIGURATION['N_DENSE_2'], activation = "relu", kernel_regularizer = L2(REGULARIZATION_RATE)),
    BatchNormalization(),

    Dense(3, activation = "softmax"),
    
    
    
]
)
lenet_model.summary()

## <b> Training <b>
    
    

In [ ]:
loss_function=tf.keras.losses.CategoricalCrossentropy(
  
)


In [ ]:
metrics=[CategoricalAccuracy(name="accuracy"),TopKCategoricalAccuracy(k=2,name="top_K_accuracy")]

In [ ]:
lenet_model.compile(
    optimizer=Adam(learning_rate=CONFIGURATION['LEARNING_RATE']),
    loss=loss_function,
    metrics=metrics
)


In [ ]:
history=lenet_model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=CONFIGURATION['EPOCHS'],
    verbose=1
)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title("Model_loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train_loss","val_loss"])
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

plt.title("Model_loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train_loss","val_loss"])
plt.show()

In [ ]:
lenet_model.evaluate(validation_dataset)

### <b> Testing </b>